In [ ]:
import os
import shutil
from tqdm import tqdm
from stardist.models import StarDist2D
import tifffile
import re
from csbdeep.utils import normalize
import cv2
import matplotlib.pyplot as plt
import random

import numpy as np
import pandas as pd

from PIL import Image
from skimage.measure import block_reduce

### Les données d'entrées

# dossier contenant les tuiles
DATA_DIR="/"
# dossier contennat lame entiere 
IMAGE_DIR='/'
# dossier avec tuile à supprimer 
SUP_DIR="/"
#dossier contenant les labels
LABEL_DIR="/"

if not os.path.exists(SUP_DIR):
    os.makedirs(SUP_DIR)

# Initialisation stardist 
StarDist2D.from_pretrained() 
HE_model = StarDist2D.from_pretrained('2D_versatile_he')

In [ ]:
def is_purple(crop, purple_threshold = 100 * 2, purple_scale_size = 15) -> bool:

    """
    Determines if a given portion of an image is purple.

    Args:
        crop: Portion of the image to check for being purple.
        purple_threshold: Number of purple points for region to be considered purple.
        purple_scale_size: Scalar to use for reducing image to check for purple.

    Returns:
        A boolean representing whether the image is purple or not.
    """
    crop = np.array(crop)
    block_size = (crop.shape[0] // purple_scale_size, crop.shape[1] // purple_scale_size, 1)
    pooled = block_reduce(image=crop, block_size=block_size, func=np.average)

    # Calculate boolean arrays for determining if portion is purple.
    r, g, b = pooled[..., 0], pooled[..., 1], pooled[..., 2]
    cond1 = r > g - 10
    cond2 = b > g - 10
    cond3 = ((r + b) / 2) > g + 20

    # Find the indexes of pooled satisfying all 3 conditions.
    pooled = pooled[cond1 & cond2 & cond3]
    num_purple = pooled.shape[0]

    return num_purple > purple_threshold


In [ ]:
#Vérifie chaque tuile de chaque lame et copie la tuile s'il n'y a pas suffisamment de tissus
for dossier in tqdm(os.listdir(DATA_DIR)):

    print(dossier)

    for fichier in tqdm(os.listdir(DATA_DIR+ dossier)):
        crop = Image.open(os.path.join(DATA_DIR,dossier, fichier))
        #print(is_purple(crop))
        if is_purple(crop):
            chemin_source = os.path.join(DATA_DIR+ dossier, fichier)
            chemin_destination = os.path.join(SUP_DIR, fichier)
            shutil.copy(chemin_source, chemin_destination)

In [ ]:
#Obtions supplémentaire : vérifier le nombre de noyaux par tuile et copie dans le dossier à supprimer s'il n'y a pas assez de noyaux ( étape longue)

for dossier in tqdm(os.listdir(DATA_DIR)):
    
    print(dossier)
    
        
    nom_fichier=dossier+'.svs'

    HE_img_path=IMAGE_DIR+nom_fichier
    HE_img=tifffile.imread(HE_img_path)

    for fichier in os.listdir(DATA_DIR+ dossier):
        print(fichier)
        resultats = re.search(r'\[d=(\d+),x=(\d+),y=(\d+),w=(\d+),h=(\d+)\]', fichier)
        x = int(resultats.group(2))
        y = int(resultats.group(3))
        w = int(resultats.group(4))
        h = int(resultats.group(5))   

        roi = HE_img[y:y+h, x:x+w]     

        he_labels, he_details = HE_model.predict_instances(normalize(roi))
        coordinates = he_details["points"]
        num_coordinates = len(coordinates)
        if num_coordinates<25:
            chemin_source = os.path.join(DATA_DIR+ dossier, fichier)
            chemin_destination = os.path.join(SUP_DIR, fichier)
            shutil.copy(chemin_source, chemin_destination)



In [ ]:
#Supprime les tuiles présentent dans le dossier à supprimer ainsi que le label associé ( conseil de vérifier le dossier à supprimer afin de ne pas éliminer les tuiles aynat du tissus)
for dossier in tqdm(os.listdir(DATA_DIR)):
    
    # Obtenez la liste des fichiers dans les deux dossiers
    
    fichiers_dossier_a = set(os.listdir(DATA_DIR+dossier))
    fichiers_dossier_l = os.listdir(LABEL_DIR+dossier)
    fichiers_modifies = [fichier.replace('.png', '.tif') for fichier in fichiers_dossier_l]

    fichiers_dossier_b = set(os.listdir(SUP_DIR))

    # Identifiez les fichiers communs aux deux dossiers
    fichiers_communs_i = fichiers_dossier_a.intersection(fichiers_dossier_b)
    fichiers_communs_l = set(fichiers_modifies).intersection(fichiers_dossier_b)

    # Supprimez les fichiers communs
    for fichier in fichiers_communs_i:
        chemin_fichier_a = os.path.join(DATA_DIR+dossier, fichier)
        chemin_fichier_b = os.path.join(SUP_DIR, fichier)

        try:
            os.remove(chemin_fichier_a)
            #print(f'Fichier supprimé: {fichier}')
        except OSError as e:
            print(f'Erreur lors de la suppression de {fichier}: {e}')

    for fichier in fichiers_communs_l:
        #print(fichier)
        f=fichier.split(".")
        nom=f[0]+".png"
        chemin_fichier_a = os.path.join(LABEL_DIR+dossier, nom)
        chemin_fichier_b = os.path.join(SUP_DIR, fichier)

        try:
            os.remove(chemin_fichier_a)
            #print(f'Fichier supprimé: {nom}')
        except OSError as e:
            print(f'Erreur lors de la suppression de {nom}: {e}')

